# Random forest

Семинар основан на <a href="https://habr.com/en/company/ods/blog/324402/">материале</a> ODS

In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

from scipy.special import binom
from IPython.display import Image
from matplotlib import pyplot as plt

from sklearn.ensemble import BaggingRegressor, BaggingClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits as load
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, train_test_split

%matplotlib inline

## Bootstrap

<img src='img/bootstrap.png'>

** Важно! **
    - Бутстрепная выборка имеет такой же размер, что и исходная
    - Генерация с повторениями

----------

## BAGGING(Bootstrap AGGregetING)

## $$a_{Bagging}(x) = \frac{1}{M}\sum_{i=1}^M b_i(x)$$

$b_i(x)$ - обучен на бутстреп-выборке $X^i$

<img src='img/bagging.png'>

In [3]:
iris = load()
X = iris.data
y = iris.target

Качество классификации одним решающим деревом:

In [4]:
d3 = DecisionTreeClassifier() # Обычное решающее дерево

print("Decision tree:", cross_val_score(d3, X, y).mean())

Decision tree: 0.7641195020655038


Качество бэггинга над решающими деревьями:

In [5]:
print("Bagging:", cross_val_score(BaggingClassifier(d3), X, y).mean())

Bagging: 0.8641988222075007


- Какой недостаток есть у деревьев?
- Как bagging борется с этим недостатком?
- Как можно улучшить качество?

Теперь при построении каждого узла будем отбирать случайные max_features признаков и искать информативное разбиение только по одному из них:

In [6]:
f = X.shape[1]
rnd_d3 = DecisionTreeClassifier(max_features=int(f ** 0.5)) # Решающее дерево с рандомизацией в сплитах

print("Randomized Bagging:", cross_val_score(BaggingClassifier(rnd_d3), X, y).mean())

Randomized Bagging: 0.8848364802374699


----------

## Random Forest

##### Алгоритм построения случайного леса из $N$ деревьев

Для каждого $n = 1..N$:

Сгенерировать выборку $X_n$ с помощью бутстрэпа;
Построить решающее дерево $b_n$ по выборке $X_n$:
    1. по заданному критерию мы выбираем лучший признак, делаем разбиение в дереве по нему и так до исчерпания выборки
    2. дерево строится, пока в каждом листе не более $n_{min}$ объектов или пока не достигнем определенной высоты дерева
    3. при каждом разбиении сначала выбирается $m$ случайных признаков из $n$ исходных, и оптимальное разделение выборки ищется только среди них.

Итоговый классификатор:
    $$ a(x) = \dfrac{1}{N} \sum_{i=1}^{N} b_i(x)$$

$m$ советуют выбирать равным:
- $\sqrt{n}$ для классификации
- $\dfrac{n}{3}$ для регрессии

----------

## Hand made Random Forest

**Необходимо реализовать класс `RandomForest`** (для решения задачи классификации)

**Спецификация:**
- конструктор содержит следующие параметры: 
    - `num_trees` - количество деревьев в лесе;
    - `max_depth` - максимальная глубина дерева (по умолчанию - `None`); 
    - `max_features` - количество признаков, принимаемое к рассмотрению при разбиении (аналогичный параметр есть в sklearn имплементации) (по умолчанию - `None`); 
    - `criterion` - критерий разбиения (для классификации - 'gini' или 'entropy', по умолчанию - 'gini'); 
    
- класс имеет методы `fit` и `predict`;
- метод `fit` принимает матрицу объектов `X` и вектор ответов `y` (объекты `numpy.ndarray`) и возвращает экземпляр класса
    `RandomForest`, представляющий собой Random Forest, обученный по выборке `(X, y)` с учётом заданных в конструкторе параметров; 
- метод `predict` принимает матрицу объектов и возвращает вектор предсказанных ответов;

In [10]:
def bootstrap(X, y, size):
    # Implement random sampling here
    sample_X, sample_y = <ваш код>
    return sample_X, sample_y

In [11]:
class RandomForest:
    def __init__(self, num_trees, max_depth=None, max_features=None, criterion='gini'):
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.criterion = criterion
        self.trees = []
    
    def fit(self, X_train, y_train):
        '''
        Create trees here, using bootstrap.
        '''
        <ваш код>
            
        return self
    
    def predict(self, X_test):
        '''
        Predict the label here using your grown trees.
        '''
        y_pred = np.zeros(X_test.shape[0])
        <ваш код>
        
        return y_pred

#### Тестирование

Разделим датасет ирисов на обучающую и тестовую выборку

In [12]:
iris = load()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Обучим наш Random Forest

In [13]:
num_trees = 10
max_depth = 10
max_features = 'sqrt'
criterion = 'gini'

clf = RandomForest(num_trees=num_trees, max_depth=max_depth, max_features=max_features, criterion=criterion)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [14]:
accuracy_score(y_pred, y_test)

0.9444444444444444

Обучим DecisionTreeClassifier

In [15]:
max_depth = 10
max_features = None
criterion = 'gini'

clf = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features, criterion=criterion)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [16]:
accuracy_score(y_pred, y_test)

0.85

----------

### 1 Предложить способы оптимизации алгоритма случайного леса (например стэкинг)

### 2 Сравнить с классом ExtraTreesClassifier sklearn

### 3 Отобразить признаки на гистограме по убыванияю важности (воспользоваться свойствами estimators_ и feature_importances_)